# WiNDC National Model

You will need to point to the correct locations for a few objects, listed below

In [1]:
parameter_path = raw"C:\Users\mphillipson\Documents\Modeling\MPSGE_dev_testing\nationaldata_julia\DAAData.jld2"
set_path = raw"C:\Users\mphillipson\Documents\Modeling\MPSGE_dev_testing\nationaldata_julia\Indices.jld2";

In [2]:
using MPSGE_MP

using CSV, DataFrames, JLD2

using JuMP

In [3]:
P= load(parameter_path)["data"] # load in date from saved Notebook output Dict, named P
S= load(set_path)["data"] # load in date from saved Notebook output Dict, named P


I = [i for i∈S[:i] if i∉[:use,:oth]]
J = [i for i∈S[:j] if i∉[:use,:oth]]
VA = [va for va∈S[:va] if va!=:othtax]
FD = S[:fd]
TS = S[:ts]
YR = S[:yr]
M = S[:m]

a_0 = P[:a_0]
id_0 = P[:id_0]
ys_0 = P[:ys_0]
tm_0 = P[:tm_0]
va_0 = P[:va_0]
md_0 = P[:md_0]
fd_0 = P[:fd_0]
m_0 = P[:m_0]
ty_0 = P[:ty_0]
ms_0 = P[:ms_0]
bopdef_0 = P[:bopdef_0]
x_0 = P[:x_0]
ta_0 = P[:ta_0]
#s_0 = P[:s_0]
fs_0 = P[:fs_0]
y_0 = P[:y_0];



1;

In [13]:
yr = Symbol(2017)

WiNnat = MPSGEModel()

#y_ = [j for j∈J if sum(ys_0[yr,j,i] for i∈I) !=0]
#a_ = [i_ for i_∈I if a_0[yr,i_]!=0]


@parameters(WiNnat, begin
    ta, ta_0[yr,J], (index = [J],)
    ty, ty_0[yr,J], (index = [J],)
    tm, tm_0[yr,J], (index = [J],)
end)

@sectors(WiNnat,begin
    Y, (index = [J], description = "Sectoral Production")
    A, (index = [I], description = "Armington Supply")
    MS, (index = [M],description = "Margin Supply")
end)

@commodities(WiNnat,begin
    PA, (index = [I],  description = "Armington Price")
    PY, (index = [J],  description = "Supply")
    PVA, (index = [VA],description = "Value-added")
    PM, (index = [M],  description = "Foreign Exachange")
    PFX
end)

@consumer(WiNnat, RA, description = "Representative Agent")

for j∈J
    @production(WiNnat, Y[j], [t=0, s = 0, va => s = 1], begin
        [@Output(PY[i],ys_0[yr,j,i], t, taxes = [Tax(RA,ty[j])]) for i∈I]... 
        [@Input(PA[i], id_0[yr,i,j], s) for i∈I]...
        [@Input(PVA[va], va_0[yr,va,j], va) for va∈VA]...
    end)
end



for m∈M
    @production(WiNnat, MS[m], [t = 0, s = 0], begin
        [@Output(PM[m], sum(ms_0[yr,i,m] for i∈I), t)]...
        [@Input(PY[i], ms_0[yr,i,m], s) for i∈I]...
    end)
end

for i∈I
    @production(WiNnat, A[i], [t = 2, s = 0, dm => s = 2], begin
        [@Output(PA[i], a_0[yr,i], t, taxes=[Tax(RA,ta[i])],reference_price=1-ta_0[yr,i])]...
        [@Output(PFX, x_0[yr,i], t)]...
        [@Input(PM[m], md_0[yr,m,i], s) for m∈M]...
        @Input(PY[i], y_0[yr,i], dm)
        @Input(PFX, m_0[yr,i], dm, taxes = [Tax(RA,tm[i])],reference_price=1+tm_0[yr,i])
    end)
end


add_demand!(WiNnat, RA,
    [ScalarDem(PA[i], fd_0[yr,i,:pce]) for i∈I],
    [
        [ScalarEndowment(PY[i], fs_0[yr,i]) for i∈I];
        [ScalarEndowment(PFX, bopdef_0[yr])];
        [ScalarEndowment(PA[i], -sum(fd_0[yr,i,xfd] for xfd∈FD if xfd!=:pce)) for i∈I];
        [ScalarEndowment(PVA[va], sum(va_0[yr,va,j] for j∈J)) for va∈VA]
    ]
);

build!(WiNnat);

# Benchmark 

In [14]:
solve!(WiNnat; :cumulative_iteration_limit => 0)

df_benchmark = generate_report(WiNnat);

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_2630.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 292

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 5.2077e-12           I 0.0e+00 2.4e-12 (zero_profit[Y(:)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.062000
Residual. . . . . . . . 5.207662e-12
Postsolved residual: 5.2077e-12


# Counterfactual

In [15]:
fix(RA,12453.896315446877)

set_value!(ta,0)
set_value!(tm,0)

solve!(WiNnat; :cumulative_iteration_limit => 10_000)

df = generate_report(WiNnat);

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_2D74.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Zero:     3 Single:     0 Double:     0
Preprocessed size   : 288

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             5.0521e+02             0.0e+00 (zero_profit[A(:pet,))
    1     1     0   288 3.7275e+01  1.0e+00    5.1e+00 (market_clearance[PFX)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 3.7275e+01           I 3.7e-01 3.7e+01 (market_clearanc)
    1     1     3     3 8.9265e-01  1.0e+00 SO 1.5e-01 8.6e-01 (market_clearanc)
    2     1     4     4 1.3717e-02  1.0e+00 SO 6.0e-02 1.1e-02 (market_clearanc)
    3     1     5     5 4.7088e-04  1.0e+00 SO 1.4e-03 4.1e-04 (market_clearanc)

In [16]:
df |>
    x -> sort(x, :margin, rev=true)

Row,var,value,margin
,GenericV…,Float64,Float64
1,PA[pip],0.765399,2.80046e-7
2,RA,12453.9,1.41681e-7
3,PY[oil],0.962449,4.92587e-10
4,PA[pet],0.81741,3.89207e-10
5,PA[amb],0.983718,2.83308e-10
6,PA[hos],0.978616,2.64208e-10
7,PA[osv],0.957477,1.43245e-10
8,PA[res],0.910259,1.39835e-10
9,PA[alt],0.878068,1.14142e-10


In [ ]:
value.(Y)